<a href="https://colab.research.google.com/github/kahhotan/Coding-Projects/blob/main/pyspark_guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyspark basic operations

In [113]:
#pip install pyspark

In [114]:
import pyspark
from pyspark.sql import SparkSession

In [115]:
#create session
spark=SparkSession.builder.appName('Practice').getOrCreate()

In [116]:
spark

In [117]:
#load dataset
df_pyspark=spark.read.csv('/content/sample_data/test1.csv')

In [118]:
type(df_pyspark)
#pyspark is sql dataframe

pyspark.sql.dataframe.DataFrame

In [119]:
df_pyspark.show()
#issue: Name, age, etc is supposed to be main header

+---------+---+----------+------+
|      _c0|_c1|       _c2|   _c3|
+---------+---+----------+------+
|     Name|age|Experience|Salary|
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [120]:
#reload dataset with .option to make first row the header
df_pyspark=spark.read.option('header','true').csv('/content/sample_data/test1.csv')

In [121]:
df_pyspark.show()
#now we have fixed the headers

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [122]:
df_pyspark.printSchema()
#issue: numerics are read as str instead of int. by default, pyspark will read all features as str unless specified

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [123]:
#reload dataset with .option and inferSchema = 'True'
df_pyspark=spark.read.option('header','true').csv('/content/sample_data/test1.csv',inferSchema='True')

In [124]:
#checking schema again
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [125]:
#alternative method of loading dataset
df_pyspark=spark.read.csv('/content/sample_data/test1.csv',header='True',inferSchema='True')

In [126]:
#exploring feature header
df_pyspark.columns

['Name', 'age', 'Experience', 'Salary']

In [127]:
#selecting specific features
df_pyspark.select('Name', 'age').show()

+---------+---+
|     Name|age|
+---------+---+
|    Krish| 31|
|Sudhanshu| 30|
|    Sunny| 29|
|     Paul| 24|
|   Harsha| 21|
|  Shubham| 23|
+---------+---+



In [128]:
#displaying dataset info
df_pyspark.describe().show()

+-------+------+------------------+-----------------+------------------+
|summary|  Name|               age|       Experience|            Salary|
+-------+------+------------------+-----------------+------------------+
|  count|     6|                 6|                6|                 6|
|   mean|  null|26.333333333333332|4.666666666666667|21333.333333333332|
| stddev|  null| 4.179314138308661|3.559026084010437| 5354.126134736337|
|    min|Harsha|                21|                1|             15000|
|    max| Sunny|                31|               10|             30000|
+-------+------+------------------+-----------------+------------------+



In [129]:
#add cols
df_pyspark=df_pyspark.withColumn('Age after 2 years',df_pyspark['Age']+2)

In [130]:
df_pyspark.show()

+---------+---+----------+------+-----------------+
|     Name|age|Experience|Salary|Age after 2 years|
+---------+---+----------+------+-----------------+
|    Krish| 31|        10| 30000|               33|
|Sudhanshu| 30|         8| 25000|               32|
|    Sunny| 29|         4| 20000|               31|
|     Paul| 24|         3| 20000|               26|
|   Harsha| 21|         1| 15000|               23|
|  Shubham| 23|         2| 18000|               25|
+---------+---+----------+------+-----------------+



In [131]:
#drop cols
df_pyspark=df_pyspark.drop('Age after 2 years')

In [132]:
df_pyspark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [133]:
#rename cols
df_pyspark=df_pyspark.withColumnRenamed('Name','New name')

In [134]:
df_pyspark.show()

+---------+---+----------+------+
| New name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



# Handling Missing Values

In [154]:
df_pyspark2=spark.read.csv('/content/sample_data/test2.csv',header='True',inferSchema='True')

In [136]:
df_pyspark2.show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
|     null|  36|      null|  null|
+---------+----+----------+------+



In [137]:
#drop name col
df_pyspark2.drop('Name').show()

+----+----------+------+
| age|Experience|Salary|
+----+----------+------+
|  31|        10| 30000|
|  30|         8| 25000|
|  29|         4| 20000|
|  24|         3| 20000|
|  21|         1| 15000|
|  23|         2| 18000|
|null|      null| 40000|
|  34|        10| 38000|
|  36|      null|  null|
+----+----------+------+



In [138]:
df_pyspark2.na.drop().show()
#if .drop has no parameters, any row with null value will be dropped

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [139]:
#any/all parameter for .drop
df_pyspark2.na.drop(how='all').show()
#any drops row with any null, all only drops row with all null

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
|     null|  36|      null|  null|
+---------+----+----------+------+



In [140]:
#thresh, sets limit of non-null which results in row being kept
df_pyspark2.na.drop(how='any', thresh=2).show()
#thresh=2 keeps row with at least 2 non-null

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
+---------+----+----------+------+



In [141]:
#subset, drops row with null value from specific cols
df_pyspark2.na.drop(how='any', subset=['Experience']).show()
#subset=['Experience'] will drop row with any null value in Experience col

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
|     null| 34|        10| 38000|
+---------+---+----------+------+



In [146]:
#filling missing values
df_pyspark2.na.fill('Missing values').show()

+--------------+--------------+--------------+--------------+
|          Name|           age|    Experience|        Salary|
+--------------+--------------+--------------+--------------+
|         Krish|            31|            10|         30000|
|     Sudhanshu|            30|             8|         25000|
|         Sunny|            29|             4|         20000|
|          Paul|            24|             3|         20000|
|        Harsha|            21|             1|         15000|
|       Shubham|            23|             2|         18000|
|        Mahesh|Missing values|Missing values|         40000|
|Missing values|            34|            10|         38000|
|Missing values|            36|Missing values|Missing values|
+--------------+--------------+--------------+--------------+



In [148]:
#filling missing values in specific cols
df_pyspark2.na.fill('Missing values', ['Experience','age']).show()

+---------+--------------+--------------+------+
|     Name|           age|    Experience|Salary|
+---------+--------------+--------------+------+
|    Krish|            31|            10| 30000|
|Sudhanshu|            30|             8| 25000|
|    Sunny|            29|             4| 20000|
|     Paul|            24|             3| 20000|
|   Harsha|            21|             1| 15000|
|  Shubham|            23|             2| 18000|
|   Mahesh|Missing values|Missing values| 40000|
|     null|            34|            10| 38000|
|     null|            36|Missing values|  null|
+---------+--------------+--------------+------+



In [150]:
#imputation
from pyspark .ml.feature import Imputer

In [152]:
imputer = Imputer(
    inputCols=['age', 'Experience', 'Salary'],
    outputCols=["{}_imputed".format(c) for c in ['age', 'Experience', 'Salary']]
    ).setStrategy("mean")
#{}_imputed.format(c) creates new imputed cols in the format specified in c

In [155]:
imputer.fit(df_pyspark2).transform(df_pyspark2).show()

+---------+----+----------+------+-----------+------------------+--------------+
|     Name| age|Experience|Salary|age_imputed|Experience_imputed|Salary_imputed|
+---------+----+----------+------+-----------+------------------+--------------+
|    Krish|  31|        10| 30000|         31|                10|         30000|
|Sudhanshu|  30|         8| 25000|         30|                 8|         25000|
|    Sunny|  29|         4| 20000|         29|                 4|         20000|
|     Paul|  24|         3| 20000|         24|                 3|         20000|
|   Harsha|  21|         1| 15000|         21|                 1|         15000|
|  Shubham|  23|         2| 18000|         23|                 2|         18000|
|   Mahesh|null|      null| 40000|         28|                 5|         40000|
|     null|  34|        10| 38000|         34|                10|         38000|
|     null|  36|      null|  null|         36|                 5|         25750|
+---------+----+----------+-